In [57]:
import geopandas as gp
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from pprint import pprint
import os
import glob
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.express as px #if using plotly
import folium
import warnings
import uuid
from sklearn.preprocessing import StandardScaler

In [58]:
ca_state = gp.read_file("data/CA_State_TIGER2016.shp")  # CA state
ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_bg = gp.read_file("data/ca_bg_clean.shp")  # CA block groups, cleaned

In [59]:
# just CA flares
ca_flares = gp.read_file("data/ca_flares_clean.shp")
print(f'Flares found: {len(ca_flares)}')

# CA block groups merged with EJscreen data
ca_bg_joined = gp.read_file("data/ca_bg_joined_clean.shp")

# update col name for correct area b/c it gets saved as a truncated version when saved as a shapefile out of the 
# main data cleaning notebook
ca_bg_joined.rename(columns={'shape_ar_1':'shape_area_new'}, inplace=True)  # old:new.
print(f'BGs found: {len(ca_bg_joined)}')

Flares found: 117
BGs found: 25607


In [60]:
# not all flares have a unique identifier — both CatalogID and ID columns have missing values
# so I create a new col "flare_id" of IDs

# generate random integer IDs
ids = np.random.randint(100000, 999999, size=len(ca_flares))

# convert integer IDs to unique string identifiers
id_str = [str(uuid.uuid4())[:8] + str(i) for i in ids]

# add the new column to the DataFrame
ca_flares['flare_id'] = id_str

In [61]:
# There are three BGs that seem to just be aquatic buffers around the actual county land.
# Dropping them for now 
# IDs: 60839900000, 61119901000, 60379902000
# Explore if needed: ca_bg_joined.explore()

ids_to_drop = [60839900000, 61119901000, 60379902000]

# Drop the rows with those IDs
ca_bg_joined = ca_bg_joined[~ca_bg_joined['ID'].isin(ids_to_drop)]

In [62]:
# set common crs for project
# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 

ca_state = ca_state.to_crs(meters_crs)
ca_counties = ca_counties.to_crs(meters_crs)
ca_bg = ca_bg.to_crs(meters_crs)

### 1) Which specific flares are the most impactful? 

Exploratory question; no hypothesis testing

EJ index = (The Environmental Indicator Percentile for Block Group) x (Demographic Index for Block Group)
  
a) Determine flaring impact metric. 

Something like: buffer_population x EJ Index x flare volume  

How best to normalize these different units? Use percentiles compared to rest of CA? 

b) Calculate impact per flare  

c) Visualize top_x flares by impact  


In [63]:
# set col list for BCM_avg calculation
col_list = ['BCM_2012','BCM_2013','BCM_2014','BCM_2015','BCM_2016','BCM_2017',
            'BCM_2018','BCM_2019','BCM_2020','BCM_2021']

# add new column for average BCM across all years
ca_flares['BCM_avg'] = ca_flares[col_list].mean(axis=1) 

In [64]:
#ca_bg_joined.rename(columns={'ID':'bg_id'}, inplace=True)  # old:new.

In [65]:
buffer_size = 2000

ca_bg_joined = ca_bg_joined[['bg_id', 'CNTY_NAME', 'Shape_Area', 'ACSTOTPOP', 'MINORPOP', 'D_PM25_2', 'shape_area_new', 'geometry']]

#flares_df = set_geometry_buffer(flares_df, buffer_size)
buffer_col = f"buffer_{buffer_size}m"
ca_flares[buffer_col] = ca_flares['geometry'].buffer(distance=buffer_size)

ca_flares = ca_flares.set_geometry(buffer_col)

temp = ca_flares.unary_union
all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=ca_flares.crs)  # convert back to geodf for processing

intersect = gp.overlay(ca_flares, ca_bg_joined, how='intersection')  # could look at keep_geom=False

In [66]:
intersect = intersect[['flare_id', 'bg_id', 'CNTY_NAME','flare_cate', 
        'BCM_2012', 'BCM_2013', 'BCM_2014', 'BCM_2015', 'BCM_2016','BCM_2017',
        'BCM_2018', 'BCM_2019','BCM_2020','BCM_2021','BCM_avg',
        'ACSTOTPOP', 'MINORPOP', 'D_PM25_2', 'shape_area_new', 'geometry']]

intersect.rename(columns={'geometry':'intersect_geom'}, inplace=True)  # old:new.
intersect = intersect.set_geometry('intersect_geom')  # set to the buffers rather than the points

In [67]:
# Create new 'area' column for the areas of the intersections
intersect['intersect_area'] = intersect.area

# Calculate the proportion of each block group within the buffer zone
intersect['intersect_prop'] = intersect['intersect_area'] / intersect['shape_area_new']

In [68]:
len(intersect)

501

In [69]:
intersect['flare_id'].nunique()

117

In [70]:
# Apply the proportion to each demographic variable to find counts by variable
demo_vars = ['ACSTOTPOP', 'MINORPOP']
for var in demo_vars:
    intersect[var + '_intersect_count'] = intersect[var] * intersect['intersect_prop']

# find overall proportions for each demo var by dividing var count by respective total population    
for var in demo_vars:
    intersect[var + '_bg_totprop'] = intersect[var] / intersect['ACSTOTPOP']

In [71]:
intersect.sample(1)

,flare_id,bg_id,CNTY_NAME,flare_cate,BCM_2012,BCM_2013,BCM_2014,BCM_2015,BCM_2016,BCM_2017,...,MINORPOP,D_PM25_2,shape_area_new,intersect_geom,intersect_area,intersect_prop,ACSTOTPOP_intersect_count,MINORPOP_intersect_count,ACSTOTPOP_bg_totprop,MINORPOP_bg_totprop
233,df37b17c865269,60133552013,Contra Costa County,flares_upstream,NaN,NaN,NaN,NaN,NaN,NaN,...,2590,43.142735,2.345994e+06,"POLYGON ((-171383.760 -553.799, -171446.729 -4...",1.561565e+06,0.665631,1987.573061,1723.983332,1.0,0.867381


In [72]:
# Define the variables to standardize
varlist = ['BCM_avg', 'D_PM25_2', 'ACSTOTPOP_intersect_count']

# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(intersect[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_norm' for var in varlist])

# Concatenate the new variables with the original DataFrame
intersect = pd.concat([intersect, intersect_norm], axis=1)


In [73]:
intersect.sample(1)

,flare_id,bg_id,CNTY_NAME,flare_cate,BCM_2012,BCM_2013,BCM_2014,BCM_2015,BCM_2016,BCM_2017,...,intersect_geom,intersect_area,intersect_prop,ACSTOTPOP_intersect_count,MINORPOP_intersect_count,ACSTOTPOP_bg_totprop,MINORPOP_bg_totprop,BCM_avg_norm,D_PM25_2_norm,ACSTOTPOP_intersect_count_norm
116,2c608047180440,60290033042,Kern County,flares_upstream,0.000681,0.001133,0.000764,0.000812,0.000189,NaN,...,"POLYGON ((34419.507 -286221.810, 34377.197 -28...",1.164332e+07,0.011068,11.510595,3.087938,1.0,0.268269,-0.306486,-0.551725,-0.778068


In [74]:
# Define the weights for each variable
bcm_weight = 0
pm25_weight = 1
pop_weight = 0

In [75]:
# Calculate the weighted variables
intersect['BCM_weighted'] = intersect['BCM_avg_norm'] * bcm_weight
intersect['D_PM25_2_weighted'] = intersect['D_PM25_2_norm'] * pm25_weight
intersect['ACSTOTPOP_weighted'] = intersect['ACSTOTPOP_intersect_count_norm'] * pop_weight

# Define the variables to sum
varlist_weighted = ['BCM_weighted', 'D_PM25_2_weighted', 'ACSTOTPOP_weighted']

# Group the block groups by flare ID and sum the weighted variables
ca_flares_merged = intersect.groupby('flare_id')[varlist_weighted].sum()

# Calculate the impact metric as the sum of the weighted variables
ca_flares_merged['impact_metric'] = ca_flares_merged[varlist_weighted].sum(axis=1)

# Sort the flares by impact metric in descending order
ca_flares_merged = ca_flares_merged.sort_values('impact_metric', ascending=False)

In [76]:
# Show the results
ca_flares_merged.head()

,BCM_weighted,D_PM25_2_weighted,ACSTOTPOP_weighted,impact_metric
flare_id,,,,
f587a499839433,0.0,26.066224,0.0,26.066224
3f86075a732302,0.0,20.961599,0.0,20.961599
52304be9597496,0.0,18.747027,0.0,18.747027
6a58ee09908071,0.0,18.142994,0.0,18.142994
0d0d44ae981012,0.0,18.124858,0.0,18.124858


In [21]:
# Define the variables to sum
varlist_norm = ['BCM_avg_norm', 'D_PM25_2_norm', 'ACSTOTPOP_intersect_count_norm']

# Group the block groups by flare ID and sum the standardized variables
ca_flares_merged = intersect.groupby('flare_id')[varlist_norm].sum()

# Calculate the impact metric as the sum of the standardized variables
ca_flares_merged['impact_metric'] = ca_flares_merged[varlist_norm].sum(axis=1)

# Sort the flares by impact metric in descending order
ca_flares_merged = ca_flares_merged.sort_values('impact_metric', ascending=False)

In [80]:
df_final = pd.merge(ca_flares_merged, intersect, on='flare_id')

In [81]:
df_final.sample(1)

,flare_id,BCM_weighted_x,D_PM25_2_weighted_x,ACSTOTPOP_weighted_x,impact_metric,bg_id,CNTY_NAME,flare_cate,BCM_2012,BCM_2013,...,ACSTOTPOP_intersect_count,MINORPOP_intersect_count,ACSTOTPOP_bg_totprop,MINORPOP_bg_totprop,BCM_avg_norm,D_PM25_2_norm,ACSTOTPOP_intersect_count_norm,BCM_weighted_y,D_PM25_2_weighted_y,ACSTOTPOP_weighted_y
214,3c5686fb336123,0.0,1.260038,0.0,1.260038,60374034033,Los Angeles County,flares_upstream,0.0,0.0,...,211.40019,194.714729,1.0,0.921072,-0.497429,0.129697,-0.481282,-0.0,0.129697,-0.0


In [24]:
df_final['geometry'].dtype

In [25]:
invalid_geoms = df_final[~df_final.is_valid]

In [26]:
invalid_geoms

,flare_id,BCM_avg_norm,D_PM25_2_norm,ACSTOTPOP_intersect_count_norm,impact_metric,Country,ISO Code,Catalog ID,id #,Latitude,...,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,BCM_avg,buffer_2000m


In [27]:
# save for use in tableau
df_final.to_csv(F"data/df_impactmetric_csv.csv", index=False)

In [28]:
# # check for missing values
# missing_values = df_final.isnull().sum()

# # filter columns with missing values
# missing_cols = missing_values[missing_values > 0]

# # print column names and number of missing values
# for col in missing_cols.index:
#     print(f"Column '{col}' has {missing_cols[col]} missing values")

In [29]:
# no_missing = missing_values[missing_values == 0]
# # print column names and number of missing values
# for col in no_missing.index:
#     print(f"Column '{col}' has {no_missing[col]} missing values")

In [56]:
df_final.sample(2)

,flare_id,BCM_avg_norm,D_PM25_2_norm,ACSTOTPOP_intersect_count_norm,impact_metric,Country,ISO Code,Catalog ID,id #,Latitude,...,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,BCM_avg,buffer_2000m
95,c7f97b07639063,-1.516616,-0.237028,-2.359830,-4.113474,United States,None,None,NaN,35.023168,...,12540,None,A,21061589620,79574760,+35.3466288,-118.7295064,POINT (94120.548 -332103.299),0.000143,"POLYGON ((96120.548 -332103.299, 96110.918 -33..."
51,eb98bb22116946,0.153482,-1.205105,-0.683838,-1.735461,United States,None,None,NaN,34.322666,...,37100,None,A,4773390489,945942791,+34.3587415,-119.1331432,POINT (60324.969 -410161.261),0.002040,"POLYGON ((62324.969 -410161.261, 62315.338 -41..."


In [49]:
for_map = df_final[['flare_id', 'BCM_avg', 'D_PM25_2', 'ACSTOTPOP_intersect_count', 'impact_metric', 'buffer_2000m']]
for_map = gp.GeoDataFrame(for_map, geometry='buffer_2000m', crs=meters_crs)

for_map.to_file("data/df_impactmetric_shp.shp", driver='ESRI Shapefile')

KeyError: "['D_PM25_2', 'ACSTOTPOP_intersect_count'] not in index"

In [ ]:
# # display the top ten flares by impact metric
# top_ten = for_map.nlargest(10, 'impact_metric')
# top_ten = top_ten.set_geometry('buffer_2000m')

In [50]:
#top_ten

In [51]:
# check for missing values
missing_values = for_map.isnull().sum()

# filter columns with missing values
missing_cols = missing_values[missing_values > 0]

# print column names and number of missing values
for col in missing_cols.index:
    print(f"Column '{col}' has {missing_cols[col]} missing values")

In [52]:
no_missing = missing_values[missing_values == 0]
# print column names and number of missing values
for col in no_missing.index:
    print(f"Column '{col}' has {no_missing[col]} missing values")

Column 'flare_id' has 0 missing values
Column 'BCM_avg_norm' has 0 missing values
Column 'D_PM25_2_norm' has 0 missing values
Column 'ACSTOTPOP_intersect_count_norm' has 0 missing values
Column 'impact_metric' has 0 missing values
Column 'buffer_2000m' has 0 missing values


## Folium Mapping

In [53]:

# Define the color scale and number of bins
color_scale = 'Reds'
num_bins = 10

# Create a map centered on the first flare
# Create a folium map with a center location
m = folium.Map(location=[38.377158,-121.645792], zoom_start=6, tiles=None,overlay=False)  #start w lat/long roughly in center of CA
base_map = folium.FeatureGroup(name='Base map', overlay=True, control=False)
folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
base_map.add_to(m)




In [54]:
# Create a choropleth layer
folium.Choropleth(
    geo_data=for_map,
    name='Impact Metric',
    data=for_map,
    columns=['flare_id', 'BCM_avg', 'D_PM25_2', 
             'ACSTOTPOP_intersect_count', 'impact_metric', 'buffer_2000m'],
    key_on='feature.properties.flare_id',
    fill_color=color_scale,
    fill_opacity=0.7,
    line_opacity=0.2,
    bins=num_bins,
    legend_name='Impact Metric'
).add_to(m)

KeyError: 'BCM_avg'

In [55]:
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#999999', 
                                'color':'#999999', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = for_map,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['flare_id', 'BCM_avg_norm', 'D_PM25_2_norm', 
             'ACSTOTPOP_intersect_count_norm', 'impact_metric'],# 'D_PM25_2', 'ACSTOTPOP', 'MINORPOP','shape_area_new', 'intersect_prop', 'intersect_area', 'MINORPOP_bg_totprop'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

In [47]:
# add hover functionality as child to map, add layering, display map
m.add_child(NIL)
m.keep_in_front(NIL)
folium.LayerControl().add_to(m)

In [48]:
# Display the map
m